# Connecticut Scraping 

In [1]:
# set up 
import pandas as pd 
import requests 
import bs4
import numpy as np
import time 
import re
from tika import parser
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from io import BytesIO
from urllib.request import urlopen

from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

%xmode Minimal

Exception reporting mode: Minimal


In [1]:
# # set headers
# header_list = {'User-Agent': REDACTED,
#               'Accept-Language': 'en-US,en;q=0.9'}

In [32]:
# state - UPDATE EACH TIME
state = 'connecticut'

In [33]:
# review of robots.txt - no restrictions

In [34]:
# test = pd.read_csv('colorado_2023.csv')
# test.head()

## Scrape Links 
`Beautiful Soup`

### Test on one link

In [35]:
# access webpage
link = 'https://portal.ct.gov/Office-of-the-Governor/News?SearchKeyword=&Month=by+Month&Year=2023&page=2'
server_response = requests.get(link, header_list)
server_response

<Response [200]>

In [36]:
# Parse and isolate content
soup = bs4.BeautifulSoup(server_response.content, features="html.parser")
# soup
results = soup.find('ul', class_ = 'list--desc').find_all('li')
# results

# title = [i.find('a').text for i in results]
# title_clean = [i.replace(u'\n', u'') for i in title]
# title_clean = [i.replace(u'\r', u'') for i in title_clean]
# title_clean = [i.strip() for i in title_clean]
# title_clean
# title_clean
date = [i.find('span', class_ = 'date').text for i in results]
# date

['11/22/2023',
 '11/21/2023',
 '11/21/2023',
 '11/16/2023',
 '11/15/2023',
 '11/14/2023',
 '11/13/2023',
 '11/13/2023',
 '11/10/2023',
 '11/9/2023',
 '11/9/2023',
 '11/8/2023',
 '11/6/2023',
 '11/6/2023',
 '11/2/2023']

### Scrape all links

In [37]:
# define webpage scraping 
def scrape_connecticut_links(website, headers):
    server_response = requests.get(website, headers)
    soup = bs4.BeautifulSoup(server_response.content, features="html.parser")
    results = soup.find('ul', class_ = 'list--desc').find_all('li')
    results_list = [{'title': i.find('a').text,
                 'date': i.find('span', class_ = 'date').text, 
                'link': i.find('a')['href']} for i in results]
    df = pd.DataFrame.from_dict(results_list)
    df.title = [i.replace(u'\n', u'') for i in df.title]
    df.title = [i.replace(u'\r', u'') for i in df.title]
    df.title = [i.strip() for i in df.title]
    df['content'] = np.nan
    return df

In [38]:
scrape_connecticut_links('https://portal.ct.gov/Office-of-the-Governor/News?SearchKeyword=&Month=by+Month&Year=2023&page=2', header_list)

,title,date,link,content
0,Governor Lamont’s 2023 Thanksgiving Message,11/22/2023,/Office-of-the-Governor/News/Press-Releases/20...,NaN
1,Governor Lamont: Flags Lowered To Half-Staff i...,11/21/2023,/Office-of-the-Governor/News/Press-Releases/20...,NaN
2,Governor Lamont Announces New Eligibility Leve...,11/21/2023,/Office-of-the-Governor/News/Press-Releases/20...,NaN
3,Governor Lamont Urges Connecticut's Semiconduc...,11/16/2023,/Office-of-the-Governor/News/Press-Releases/20...,NaN
4,Governor Lamont Announces Connecticut Business...,11/15/2023,/Office-of-the-Governor/News/Press-Releases/20...,NaN
5,Governor Lamont Announces Plans To Nominate Da...,11/14/2023,/Office-of-the-Governor/News/Press-Releases/20...,NaN
6,Governor Lamont and Secretary Beckham Statemen...,11/13/2023,/Office-of-the-Governor/News/Press-Releases/20...,NaN
7,Governor Lamont Announces Launch of Microtrans...,11/13/2023,/Office-of-the-Governor/News/Press-Releases/20...,NaN
8,Governor Lamont Announces Q Bridge Will Be Ill...,11/10/2023,/Office-of-the-Governor/News/Press-Releases/20...,NaN
9,Governor Lamont Debuts 'Maker's Manifesto: Did...,11/9/2023,/Office-of-the-Governor/News/Press-Releases/20...,NaN


In [39]:
# try to scrape the first set of pages
# manual review for 2023 - I want pages 1-19, give or take a few links on first and last pages
df_list=[]

for i in range(1,20):
    try: 
        df = scrape_connecticut_links(f'https://portal.ct.gov/Office-of-the-Governor/News?SearchKeyword=&Month=by+Month&Year=2023&page={i}', 
                                  header_list)
        df['page_scraped'] = i 
        df_list.append(df)
    except Exception as e:
        print(f'Error: {e} at page {i}')

In [40]:
# combine into one df
combined = pd.concat(df_list, ignore_index=True)
combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 283 entries, 0 to 282
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         283 non-null    object 
 1   date          283 non-null    object 
 2   link          283 non-null    object 
 3   content       0 non-null      float64
 4   page_scraped  283 non-null    int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 11.2+ KB


In [41]:
combined.head()

,title,date,link,content,page_scraped
0,Governor Lamont Releases Draft State Digital E...,12/22/2023,/Office-of-the-Governor/News/Press-Releases/20...,NaN,1
1,Governor Lamont Announces Connecticut Income T...,12/20/2023,/Office-of-the-Governor/News/Press-Releases/20...,NaN,1
2,Governor Lamont Directs $10.7 Million in Ameri...,12/19/2023,/Office-of-the-Governor/News/Press-Releases/20...,NaN,1
3,Governor Lamont Encourages High School Seniors...,12/19/2023,/Office-of-the-Governor/News/Press-Releases/20...,NaN,1
4,Governor Lamont Announces $7.2 Million in Stat...,12/18/2023,/Office-of-the-Governor/News/Press-Releases/20...,NaN,1


In [42]:
# URL prefix was missing so let's add this to the links 
combined.link = ['https://portal.ct.gov/' + i for i in combined.link]
combined.head()

,title,date,link,content,page_scraped
0,Governor Lamont Releases Draft State Digital E...,12/22/2023,https://portal.ct.gov//Office-of-the-Governor/...,NaN,1
1,Governor Lamont Announces Connecticut Income T...,12/20/2023,https://portal.ct.gov//Office-of-the-Governor/...,NaN,1
2,Governor Lamont Directs $10.7 Million in Ameri...,12/19/2023,https://portal.ct.gov//Office-of-the-Governor/...,NaN,1
3,Governor Lamont Encourages High School Seniors...,12/19/2023,https://portal.ct.gov//Office-of-the-Governor/...,NaN,1
4,Governor Lamont Announces $7.2 Million in Stat...,12/18/2023,https://portal.ct.gov//Office-of-the-Governor/...,NaN,1


In [43]:
# save just in case
combined.to_csv(f'{state}_links.csv', index = False)

In [44]:
combined = pd.read_csv(f'{state}_links.csv')
combined.head()

,title,date,link,content,page_scraped
0,Governor Lamont Releases Draft State Digital E...,12/22/2023,https://portal.ct.gov//Office-of-the-Governor/...,NaN,1
1,Governor Lamont Announces Connecticut Income T...,12/20/2023,https://portal.ct.gov//Office-of-the-Governor/...,NaN,1
2,Governor Lamont Directs $10.7 Million in Ameri...,12/19/2023,https://portal.ct.gov//Office-of-the-Governor/...,NaN,1
3,Governor Lamont Encourages High School Seniors...,12/19/2023,https://portal.ct.gov//Office-of-the-Governor/...,NaN,1
4,Governor Lamont Announces $7.2 Million in Stat...,12/18/2023,https://portal.ct.gov//Office-of-the-Governor/...,NaN,1


## Scrape content from links
`Beautiful Soup`

### Test with one link

In [45]:
# define link scraping 
def get_connecticut_content(link, df, header_list):
    try: 
        # check if content has already been pulled
        result = df.loc[df.link == link, 'content'].notnull().all()
        if result:
            return f'content already existing for {link}' 
        else: 
            try: 
                df['content']
                server_response = requests.get(link, headers = header_list)
                soup_link = bs4.BeautifulSoup(server_response.content, features="html.parser")
                result = soup_link.find('div', class_ = 'content').find_all('p')
                string_result = [str(i.text) for i in result]
                string_result_clean = [i.replace(u'\xa0', u'') for i in string_result]
                string_result_clean = [i.replace(u'\n', u'') for i in string_result_clean]
                string_result_clean = [i.replace(u'\r', u'') for i in string_result_clean]
                result_merged = " ".join(string_result_clean)
                df.loc[df.link == link, 'content'] = result_merged
                return f"success: content added from {link}"
            except: 
                pass
    except Exception as e: 
        return f"unable to gather content from {link} due to {e}"
    #return df_updated 

In [46]:
# access webpage
link = 'https://portal.ct.gov/Office-of-the-Governor/News/Press-Releases/2023/12-2023/Governor-Lamont-Announces-Income-Tax-Rates-Go-Down-at-the-Start-of-2024'
server_response = requests.get(link, headers = header_list)
server_response

<Response [200]>

In [47]:
# parse and isolate content 
soup_link = bs4.BeautifulSoup(server_response.content, features="html.parser")
result = soup_link.find('div', class_ = 'content').find_all('p')
string_result = [str(i.text) for i in result]
string_result_clean = [i.replace(u'\xa0', u'') for i in string_result]
string_result_clean = [i.replace(u'\n', u'') for i in string_result_clean]
string_result_clean = [i.replace(u'\r', u'') for i in string_result_clean]
result_merged = " ".join(string_result_clean)
result_merged

'12/20/2023             (HARTFORD, CT) – Governor Ned Lamont today announced that three significant tax relief measures will take effect in Connecticut at the start of 2024, and among them are the largest income tax reduction ever enacted in state history, an increase in a tax credit targeting the lowest-income workers, and an expansion of exemptions on certain pension and annuity earnings to benefit seniors. They are the result of the fiscal year 2024-2025 state budget that the Connecticut General Assembly approved and Governor Lamont signed into law this summer (Public Act 23-204). In total, the three measures will reduce taxes for Connecticut taxpayers by approximately $460.3 million. Due to the fiscal guardrails and smart management of the state budget, nonpartisan analysts anticipate that revenue growth will exceed growth in fixed costs for the next several years. For example, General Fund revenue is expected to increase by $393.4 million from fiscal year 2025 to 2026 and fixed co

### Scrape content from all links

In [48]:
# test on 5 to see if it works
combined.content = np.nan

for i in combined.link[0:6]:
     get_connecticut_content(i, combined, header_list)
    
combined.head()

'success: content added from https://portal.ct.gov//Office-of-the-Governor/News/Press-Releases/2023/12-2023/Governor-Lamont-Releases-Draft-State-Digital-Equity-Plan'

'success: content added from https://portal.ct.gov//Office-of-the-Governor/News/Press-Releases/2023/12-2023/Governor-Lamont-Announces-Income-Tax-Rates-Go-Down-at-the-Start-of-2024'

'success: content added from https://portal.ct.gov//Office-of-the-Governor/News/Press-Releases/2023/12-2023/Governor-Lamont-Directs-ARPA-and-RGGI-Funds-To-Help-Low-Income-Residents-Lower-Energy-Bills'

'success: content added from https://portal.ct.gov//Office-of-the-Governor/News/Press-Releases/2023/12-2023/Governor-Lamont-Encourages-High-School-Seniors-to-Apply-for-Automatic-College-Admission-Program'

'success: content added from https://portal.ct.gov//Office-of-the-Governor/News/Press-Releases/2023/12-2023/Governor-Lamont-Announces-State-Grants-To-Support-the-Remediation-of-Blighted-Properties'

'success: content added from https://portal.ct.gov//Office-of-the-Governor/News/Press-Releases/2023/12-2023/Governor-Lamont-Announces-Thousands-of-Low-Level-Convictions-Set-To-Be-Erased-Over-the-Next-Month'

,title,date,link,content,page_scraped
0,Governor Lamont Releases Draft State Digital E...,12/22/2023,https://portal.ct.gov//Office-of-the-Governor/...,"12/22/2023 (HARTFORD, CT) – Govern...",1
1,Governor Lamont Announces Connecticut Income T...,12/20/2023,https://portal.ct.gov//Office-of-the-Governor/...,"12/20/2023 (HARTFORD, CT) – Govern...",1
2,Governor Lamont Directs $10.7 Million in Ameri...,12/19/2023,https://portal.ct.gov//Office-of-the-Governor/...,"12/19/2023 (HARTFORD, CT) – Govern...",1
3,Governor Lamont Encourages High School Seniors...,12/19/2023,https://portal.ct.gov//Office-of-the-Governor/...,"12/19/2023 (NEW BRITAIN, CT) – Gov...",1
4,Governor Lamont Announces $7.2 Million in Stat...,12/18/2023,https://portal.ct.gov//Office-of-the-Governor/...,"12/18/2023 (HARTFORD, CT) – Govern...",1


In [49]:
# do for all
for i in combined.link:
     get_connecticut_content(i, combined, header_list)

'content already existing for https://portal.ct.gov//Office-of-the-Governor/News/Press-Releases/2023/12-2023/Governor-Lamont-Releases-Draft-State-Digital-Equity-Plan'

'content already existing for https://portal.ct.gov//Office-of-the-Governor/News/Press-Releases/2023/12-2023/Governor-Lamont-Announces-Income-Tax-Rates-Go-Down-at-the-Start-of-2024'

'content already existing for https://portal.ct.gov//Office-of-the-Governor/News/Press-Releases/2023/12-2023/Governor-Lamont-Directs-ARPA-and-RGGI-Funds-To-Help-Low-Income-Residents-Lower-Energy-Bills'

'content already existing for https://portal.ct.gov//Office-of-the-Governor/News/Press-Releases/2023/12-2023/Governor-Lamont-Encourages-High-School-Seniors-to-Apply-for-Automatic-College-Admission-Program'

'content already existing for https://portal.ct.gov//Office-of-the-Governor/News/Press-Releases/2023/12-2023/Governor-Lamont-Announces-State-Grants-To-Support-the-Remediation-of-Blighted-Properties'

...

'success: content added from https://portal.ct.gov//Office-of-the-Governor/News/Press-Releases/2023/01-2023/Governor-Lamont-Announces-Global-Atlantic-Financial-Group-Is-Expanding'

'success: content added from https://portal.ct.gov//Office-of-the-Governor/News/Press-Releases/2023/01-2023/Governor-Lamont-Coronavirus-Update-January-5'

'success: content added from https://portal.ct.gov//Office-of-the-Governor/News/Press-Releases/2023/01-2023/Governor-Lamont-Directs-State-Flags-To-Half-Staff-in-Honor-of-Q'

'success: content added from https://portal.ct.gov//Office-of-the-Governor/News/Press-Releases/2023/01-2023/Governor-Lamont-Health-and-Safety-Guidance-for-Those-Attending-the-Inauguration'

'success: content added from https://portal.ct.gov//Office-of-the-Governor/News/Press-Releases/2023/01-2023/Governor-Lamont-Directs-Flags-to-Return-to-Full-Staff-Tuesday-Evening'

## Validate and Clean

In [50]:
#check that content was pulled accurately
combined[combined.content.isna()]

,title,date,link,content,page_scraped
166,Governor Lamont: Flags Lowered To Half-Staff i...,5/7/2023,https://portal.ct.gov//Office-of-the-Governor/...,NaN,12


In [55]:
# retry 166 - looks like link has extra slash 
combined.link[166]
combined.iloc[166,2] = 'https://portal.ct.gov/Office-of-the-Governor/News/Press-Releases/2023/05-2023/Governor-Lamont-Flags-Lowered-To-Half-Staff-in-Honor-of-the-Victims-in-Texas'
combined.link[166]

'https://portal.ct.gov//Office-of-the-Governor/News/Press-Releases/2023/05-2023/Governor-Lamont-Flags-Lowered-To-Half-Staff-in-Honor-of-the-Victims-in-Texas'

'https://portal.ct.gov/Office-of-the-Governor/News/Press-Releases/2023/05-2023/Governor-Lamont-Flags-Lowered-To-Half-Staff-in-Honor-of-the-Victims-in-Texas'

In [56]:
get_connecticut_content(combined.link[166], combined, header_list)

'success: content added from https://portal.ct.gov/Office-of-the-Governor/News/Press-Releases/2023/05-2023/Governor-Lamont-Flags-Lowered-To-Half-Staff-in-Honor-of-the-Victims-in-Texas'

In [57]:
# check this worked
combined[combined.content.isna()]

,title,date,link,content,page_scraped,date_clean


In [58]:
# convert date to date-time object to subset
combined['date_clean'] = pd.to_datetime(combined['date'])

In [59]:
# drop original date column 
combined_cl = combined.drop(columns=['date']).copy()

In [60]:
# subset to 2023 only 
combined_2023 = combined_cl[(combined_cl['date_clean'] >= '2023-01-01')
                     & (combined_cl['date_clean'] <= '2023-12-31')].copy()

In [61]:
print(f"pulled {str(len(combined_2023))} documents from {state} for 2023")

pulled 283 documents from connecticut for 2023


## Export 

In [62]:
# export
combined_2023.to_csv(f'{state}_2023.csv', index = False)

In [63]:
# test = pd.read_csv(f'{state}_2023.csv')
# test.head()

,title,link,content,page_scraped,date_clean
0,Governor Lamont Releases Draft State Digital E...,https://portal.ct.gov//Office-of-the-Governor/...,"12/22/2023 (HARTFORD, CT) – Govern...",1,2023-12-22
1,Governor Lamont Announces Connecticut Income T...,https://portal.ct.gov//Office-of-the-Governor/...,"12/20/2023 (HARTFORD, CT) – Govern...",1,2023-12-20
2,Governor Lamont Directs $10.7 Million in Ameri...,https://portal.ct.gov//Office-of-the-Governor/...,"12/19/2023 (HARTFORD, CT) – Govern...",1,2023-12-19
3,Governor Lamont Encourages High School Seniors...,https://portal.ct.gov//Office-of-the-Governor/...,"12/19/2023 (NEW BRITAIN, CT) – Gov...",1,2023-12-19
4,Governor Lamont Announces $7.2 Million in Stat...,https://portal.ct.gov//Office-of-the-Governor/...,"12/18/2023 (HARTFORD, CT) – Govern...",1,2023-12-18
